<a href="https://colab.research.google.com/github/andersonfurtado/AI4WEBDEV/blob/main/ch9/Movie_Recommendation_with_Sentiment_Analysis_CLI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install the necessary libraries
!pip install scikit-surprise pandas sqlalchemy cryptography textblob spacy

# Download spaCy model
!python -m spacy download en_core_web_sm

# Import the libraries
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy
import urllib.request
import zipfile
import os
from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from cryptography.fernet import Fernet
from textblob import TextBlob
import spacy

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Use environment variables to store sensitive information
DATABASE_URL = os.getenv('DATABASE_URL', 'sqlite:///movies.db')

# Encrypt the database URL (just as an example; typically encryption keys should be managed securely)
key = Fernet.generate_key()
cipher_suite = Fernet(key)
ciphered_url = cipher_suite.encrypt(DATABASE_URL.encode())

# Configure the SQLite database
engine = create_engine(DATABASE_URL)
Base = declarative_base()

# Define the ORM classes
class Movie(Base):
    __tablename__ = 'movies'
    movieId = Column(Integer, primary_key=True)
    title = Column(String)
    genres = Column(String)

class Rating(Base):
    __tablename__ = 'ratings'
    userId = Column(Integer, primary_key=True, autoincrement=True)
    movieId = Column(Integer)
    rating = Column(Float)
    timestamp = Column(Integer)

# Create the tables in the database
Base.metadata.create_all(engine)

# Configure the session
Session = sessionmaker(bind=engine)
session = Session()

# Download and unzip the MovieLens dataset
url = 'https://files.grouplens.org/datasets/movielens/ml-latest-small.zip'
urllib.request.urlretrieve(url, 'ml-latest-small.zip')

with zipfile.ZipFile('ml-latest-small.zip', 'r') as zip_ref:
    zip_ref.extractall()

# Full path to the files
movies_file = os.path.join('ml-latest-small', 'movies.csv')
ratings_file = os.path.join('ml-latest-small', 'ratings.csv')

# Load the data
movies = pd.read_csv(movies_file)
ratings = pd.read_csv(ratings_file)

# Insert the data into the database
movies.to_sql('movies', engine, if_exists='replace', index=False)
ratings.to_sql('ratings', engine, if_exists='replace', index=False)

# Prepare the data for the Surprise library
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# Split the data into training and test sets
trainset, testset = train_test_split(data, test_size=0.2)

# Train the SVD model
algo = SVD()
algo.fit(trainset)

# Function to recommend movies
def get_movie_recommendations(algo, movie_title, movies, ratings, num_recommendations=5):
    if movie_title not in movies['title'].values:
        return None
    movie_id = movies[movies['title'] == movie_title]['movieId'].values[0]
    users_who_rated_movie = ratings[ratings['movieId'] == movie_id]['userId'].unique()
    other_movie_ids = ratings[ratings['userId'].isin(users_who_rated_movie)]['movieId'].unique()
    predicted_ratings = []
    for movie in other_movie_ids:
        if movie != movie_id:
            predicted_ratings.append((movie, algo.predict(uid=0, iid=movie).est))
    predicted_ratings.sort(key=lambda x: x[1], reverse=True)
    top_n_movies = [movie_id for movie_id, rating in predicted_ratings[:num_recommendations]]
    recommended_movie_titles = movies[movies['movieId'].isin(top_n_movies)]['title'].tolist()
    return recommended_movie_titles

# Function to check if a movie title exists
def check_movie_title(title, session):
    result = session.query(Movie).filter(Movie.title.ilike(f"%{title}%")).all()
    return result

# Function for sentiment analysis
def analyze_sentiment(text):
    blob = TextBlob(text)
    return blob.sentiment

# Function for named entity recognition
def extract_entities(text):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

# Function to start the chatbot CLI
def chatbot():
    print("Welcome to the Movie Recommendation CLI!")
    while True:
        user_input = input("Enter a movie title or a message (or 'exit' to quit): ")
        if user_input.lower() == 'exit':
            break
        sentiment = analyze_sentiment(user_input)
        entities = extract_entities(user_input)
        print(f"Sentiment Analysis: Polarity = {sentiment.polarity}, Subjectivity = {sentiment.subjectivity}")
        print(f"Named Entities: {entities}")

        if sentiment.polarity < 0:
            print("It seems like you're feeling down. Here are some feel-good movie recommendations to cheer you up!")

        movie_title = user_input
        found_movies = check_movie_title(movie_title, session)
        if found_movies:
            print(f"Found {len(found_movies)} movie(s):")
            for movie in found_movies:
                print(f"- {movie.title}")
            title_to_recommend = input("Please enter the exact title for recommendations: ")
            recommendations = get_movie_recommendations(algo, title_to_recommend, movies, ratings)
            if recommendations:
                print(f"Recommendations for '{title_to_recommend}':")
                for idx, rec in enumerate(recommendations, start=1):
                    print(f"{idx}. {rec}")
            else:
                print(f"No recommendations found for '{title_to_recommend}'.")
        else:
            print(f"No movies found with title '{movie_title}'. Please try again.")

if __name__ == '__main__':
    chatbot()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357217 sha256=115552ee8214832e0725d770d2bc9f971c0424738af1179b41aa629157ae443b
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 53.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


<ipython-input-1-5efe5460d995>:35: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


Welcome to the Movie Recommendation CLI!
Enter a movie title or a message (or 'exit' to quit): great
Sentiment Analysis: Polarity = 0.8, Subjectivity = 0.75
Named Entities: []
Found 36 movie(s):
- Great Day in Harlem, A (1994)
- Great White Hype, The (1996)
- Land Before Time III: The Time of the Great Giving (1995)
- Great Race, The (1965)
- Great Escape, The (1963)
- Great Dictator, The (1940)
- Great Expectations (1998)
- Barney's Great Adventure (1998)
- Great Ziegfeld, The (1936)
- Great Mouse Detective, The (1986)
- Great Santini, The (1979)
- Great Muppet Caper, The (1981)
- Greatest Story Ever Told, The (1965)
- Great Outdoors, The (1988)
- Great Balls of Fire! (1989)
- Great Silence, The (Grande silenzio, Il) (1969)
- The Great Train Robbery (1978)
- Great Gatsby, The (1974)
- Great Expectations (1946)
- The Great Raid (2005)
- Greatest Game Ever Played, The (2005)
- Great Yokai War, The (Yôkai daisensô) (2005)
- The Great Train Robbery (1903)
- It's the Great Pumpkin, Charlie